# Strain Optimization in MEWpy

This notebook exemplifies how MEWpy may be used in strain optimization tasks.
We will consider the as goal to increase the production of succinate in E. coli in anaerobic conditions.

In [1]:
import pandas as pd
pd.set_option("max_colwidth", None)

%matplotlib inline
from cobra.io.sbml import read_sbml_model
model = read_sbml_model("models/ec/e_coli_core.xml.gz")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-11


Analyse the model's default medium

In [2]:
from mewpy.simulation.environment import Environment
import pandas as pd

medium = Environment.from_model(model).get_compounds()
env = Environment.from_compounds(medium)
env

,lb,ub
EX_co2_e,-10.0,inf
EX_glc__D_e,-10.0,inf
EX_h_e,-10.0,inf
EX_h2o_e,-10.0,inf
EX_nh4_e,-10.0,inf
EX_o2_e,-10.0,inf
EX_pi_e,-10.0,inf


Commercial solvers are better suited to address optimization tasks, such, as stain designs. As such, you might want to check the linear programming solvers available on your system:

## Optimization problem

Before running any optimization let us verify the production rate as well as the theoretic maximum rate of succinate in anaerobic conditions.
To ease the script reutilization we will define some reaction identifiers as constants:

In [3]:
BIOMASS = 'BIOMASS_Ecoli_core_w_GAM'
PRODUCT = 'EX_succ_e'
O2 = 'EX_o2_e'
GLC = 'EX_glc__D_e'

In [4]:
anaerobic = {O2: (0, 0)}

In [5]:
from mewpy.simulation import get_simulator
sim = get_simulator(model,envcond=anaerobic)
sim.FVA(reactions=[PRODUCT],format='df')

,Minimum,Maximum
Reaction ID,,
EX_succ_e,0.0,1.911


In [6]:
sim.simulate(method='pFBA').find([BIOMASS,PRODUCT])

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM,0.211663


The optimization problem requires the definition of one or more objective functions.
In the next example, we define as objective the maximization of the Biomass-Product coupled yield and the maximization of the targeted product. MEWpy makes available other optimization objectives, please refer to the documentation.

In [7]:
from mewpy.optimization.evaluation import BPCY, TargetFlux

objs = [ 
    BPCY(BIOMASS, PRODUCT),
    TargetFlux(PRODUCT)
]

We can now define the growth medium and select the type of modification.
We will begin by performing gene deletion (GKO) by selecting the appropriate problem instance, GKOProblem. 

In [8]:
from mewpy.problems import GKOProblem, GOUProblem

problem = GOUProblem(model, objs, envcond=anaerobic, candidate_max_size=6)

Other optimization methods/strategies may be used:
* Reaction deletion: *RKOProblem*
* Reaction under or over expression: *ROUProblem*
* Gene under or over expression: *GOUProblem*

A problem may include other parameters, such as the maximum number of genetic modification, a list of specitic targets, or non-targets (reactions/genes not to be modified). 

## Optimization algorithm

An optimization engine to solve the problem need to be instanciated. MEWpy uses Evolutionary Algorithms (EA) for this task.

EAs are algorithms that mimic the Darwinian evolutionary process, where a population of solutions evolves generation after generation. In the example we define a maximum of 50 generations.  

In [9]:
from mewpy.optimization import EA
ea = EA(problem, max_generations=50)

For informations on additional options you may use the `help` command

In [10]:
help(EA)

Help on function EA in module mewpy.optimization:

EA(problem, initial_population=[], max_generations=100, mp=True, visualizer=False, algorithm=None, **kwargs)
    EA running helper. Returns an instance of the EA that reflects the global user configuration settings
    such as preferred engine and algorithm.
    
    :param problem: The optimization problem.
    :param list initial_population: The EA initial population. Default [].
    :param int max_generations: The number of iterations of the EA (stopping criteria). Default globally defined.
    :param bool mp: If multiprocessing should be used. Default True.
    :param bool visualizer: If the pareto font should be displayed. Requires a graphic environment. Default False.
    
    Additional optional arguments:
    
    :param int population_size: EA population size.
    
    :returns: An instance of an EA optimizer.



The EA may contemplate aditional parameters such as the identification of a specific algorithm (e.g. SPEA2, NSGAII, NSGAIII, GDE3, etc.), multiprocessing options, initial seeding, etc.

To start the optimization process invoke the run method (Optionaly you may define if the final solutions are to be simplified or not):

In [11]:
ea.run(simplify=True)

Running NSGAII
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpte7rtgbb.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpbw_a07ga.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpcai3gtcb.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp0rx1ukb1.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpx54dt31z.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpjvtrgz76.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from

Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpi9bvom15.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp8fsr6lh2.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
: 72 rows, 190 columns, 720 nonzeros
Reading time = 0.00 seconds
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpbcvdlkeu.lp
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmprzteqv78.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp3nh_hl6h.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpuh3vhmwr.lp
Reading time = 0.00 seconds
: 72 rows, 1

: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpji_btvsp.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpsx23sipj.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpkg9qhcvm.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
: 72 rows, 190 columns, 720 nonzeros
Reading time = 0.00 seconds
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpy_tsk12q.lp
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpznaaczm4.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpb1t1ns2d.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Rea

Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpyxn9z68k.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmporlcm3qb.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpb5lqgnpo.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
   1000|  0.229993  0.841615  0.266637  0.292506  0.099232|  1.652859  7.055392  1.652859  2.241579  1.338030|
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpkr5_2k6x.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpz589ympt.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp4f2axc

Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmptzwh_nwh.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpb7g7k7h2.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp8_dxifwd.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp4vvgjmmi.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmply38imi2.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpgmco0mks.lp
Reading time = 0.00 seconds
: 72 rows, 1

...

: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpd6x4lphd.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpx1n_owd5.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpsrhra33t.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpibxd7p55.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpmeohc48g.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpwl_8_1fi.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
   

Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpr9dgc79s.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpc52pxbkn.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
   4600|  0.718930  0.918156  0.752342  0.780144  0.054124|  8.403136  8.758902  8.700666  8.652970  0.094683|
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmppu0byqc3.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp_3ridi_2.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpjcfxxx56.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp3h6hpj

Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpcrr1wb4x.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp8gquk075.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmph7hq3_5u.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmp_xhco5u4.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmphdl4y4yp.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmprlp55z95.lp
Reading time = 0.00 seconds
: 72 rows, 1

[0.7523421321470497, 8.700665974296037];{'b3870': 32, 'b2914': 2, 'b3403': 0.0625, 'b0729': 32, 'b1818': 0.0625, 'b3956': 16}  simplified to  [0.7523421321470497, 8.700665974296035];{'b2914': 2, 'b3956': 16, 'b3870': 32}
[0.7523421321470497, 8.700665974296037];{'b2285': 0.5, 'b1611': 16, 'b3870': 32, 'b2914': 2, 'b4232': 0.0625, 'b3956': 16}  simplified to  [0.7523421321470497, 8.700665974296035];{'b2914': 2, 'b3956': 16, 'b3870': 32}
[0.7523421321470497, 8.700665974296037];{'b3403': 16, 'b3870': 32, 'b2278': 0.125, 'b2914': 2, 'b1818': 0.0625, 'b3956': 16}  simplified to  [0.7523421321470497, 8.700665974296035];{'b2914': 2, 'b3956': 16, 'b3870': 32}
[0.7523421321470497, 8.700665974296037];{'b0727': 0.03125, 'b3870': 32, 'b2914': 2, 'b4232': 0.0625, 'b1818': 0.0625, 'b3956': 16}  simplified to  [0.7523421321470497, 8.700665974296035];{'b2914': 2, 'b3956': 16, 'b3870': 32}
[0.7523421321470497, 8.700665974296037];{'b2285': 0.5, 'b2463': 0.25, 'b3870': 32, 'b2914': 2, 'b1818': 0.0625, 'b3

[[0.9181564151188526, 8.533954360914194];{'b3956': 16, 'b2463': 0},
 [0.8489079919229925, 8.66502935034093];{'b2463': 0, 'b3870': 16, 'b3956': 16},
 [0.7747403052620707, 8.800405071091362];{'b2463': 0, 'b3870': 32, 'b3956': 16},
 [0.9115981881633864, 8.546575594684327];{'b2463': 0, 'b1297': 2, 'b3956': 16},
 [0.7189300114819218, 8.75890236373083];{'b3403': 0, 'b3956': 16, 'b3870': 32, 'b2914': 2},
 [0.9150663940283817, 8.403135775377892];{'b3956': 16, 'b2914': 2},
 [0.836701413447523, 8.549470033946575];{'b2914': 2, 'b3870': 16, 'b3956': 16},
 [0.8251965952132045, 8.570459156801915];{'b2914': 2, 'b2463': 0, 'b3870': 16, 'b3956': 16},
 [0.7523421321470497, 8.700665974296035];{'b2914': 2, 'b3956': 16, 'b3870': 32},
 [0.7510429616263212, 8.702947161544502];{'b3870': 32, 'b2463': 0, 'b2914': 2, 'b3956': 16}]

We can now list the set of solutions:

In [12]:
df=ea.dataframe()
df

,Modification,Size,BPCY,TargetFlux
0,"{'b3956': 16, 'b2463': 0}",2,0.918156,8.533954
1,"{'b2463': 0, 'b3870': 16, 'b3956': 16}",3,0.848908,8.665029
2,"{'b2463': 0, 'b3870': 32, 'b3956': 16}",3,0.774740,8.800405
3,"{'b2463': 0, 'b1297': 2, 'b3956': 16}",3,0.911598,8.546576
4,"{'b3403': 0, 'b3956': 16, 'b3870': 32, 'b2914': 2}",4,0.718930,8.758902
5,"{'b3956': 16, 'b2914': 2}",2,0.915066,8.403136
6,"{'b2914': 2, 'b3870': 16, 'b3956': 16}",3,0.836701,8.549470
7,"{'b2914': 2, 'b2463': 0, 'b3870': 16, 'b3956': 16}",4,0.825197,8.570459
8,"{'b2914': 2, 'b3956': 16, 'b3870': 32}",3,0.752342,8.700666
9,"{'b3870': 32, 'b2463': 0, 'b2914': 2, 'b3956': 16}",4,0.751043,8.702947


Or view the best solutions on the objective space, the so called Pareto front:

In [13]:
ea.final_population

[[0.9181564151188526, 8.533954360914194];{'b3956': 16, 'b2463': 0},
 [0.8489079919229925, 8.66502935034093];{'b2463': 0, 'b3870': 16, 'b3956': 16},
 [0.7747403052620707, 8.800405071091362];{'b2463': 0, 'b3870': 32, 'b3956': 16},
 [0.9115981881633864, 8.546575594684327];{'b2463': 0, 'b1297': 2, 'b3956': 16},
 [0.7189300114819218, 8.75890236373083];{'b3403': 0, 'b3956': 16, 'b3870': 32, 'b2914': 2},
 [0.9150663940283817, 8.403135775377892];{'b3956': 16, 'b2914': 2},
 [0.836701413447523, 8.549470033946575];{'b2914': 2, 'b3870': 16, 'b3956': 16},
 [0.8251965952132045, 8.570459156801915];{'b2914': 2, 'b2463': 0, 'b3870': 16, 'b3956': 16},
 [0.7523421321470497, 8.700665974296035];{'b2914': 2, 'b3956': 16, 'b3870': 32},
 [0.7510429616263212, 8.702947161544502];{'b3870': 32, 'b2463': 0, 'b2914': 2, 'b3956': 16}]

We can save the solutions to file:

In [14]:
df.to_csv('solutions.csv')

or performe additional analysis on the solutions by retreiving the simulator used during the optimization.
 

## Working with solutions

Let us select the first solution:

In [15]:
solution = solutions[1]
solution

NameError: name 'solutions' is not defined

The solution is converted to metabolict constraints to be applied to the model. We can access these constrains using *solution.constraints*

In [ ]:
solution.constraints

We may run phenotypic simulations with the solution using those constraints. Note that there is no need to redefine the medium as they are persistent in the simulator instance.

In [ ]:
sim = problem.simulator
res=sim.simulate(constraints=solution.constraints,method='ROOM')
res

In [ ]:
res.dataframe

In [ ]:
res.find([PRODUCT,BIOMASS])

In [ ]:
from mewpy.visualization.envelope import plot_flux_envelope

plot_flux_envelope(sim,BIOMASS,PRODUCT,constraints = solution.constraints)

## Exercice 1

Alter the notebook to run a gene over/under-expression (GOUProblem) optimization task. You may also try other optimization objectives (replacing or adding new objectives) such as `CandidateSize`, `ModificationType` or `BPCY_FVA`.

## Exercice 2

Alter the notebook to find possible genetic modifications for the increased production of ethanol (EX_etoh_e)

# Simulating user defined modifications

Genetic modifications at the gene, enzyme, transcription or regulatory levels need to be translated to the (pseudo) reaction level. This task is problem dependent and consequently requires the instantiation of a problem. If we do not intend run any optimization task, there is no need to define optimization objectives.

In [ ]:
problem = GOUProblem(model,[], envcond=anaerobic)
sim = problem.simulator

A solution for a GOUProblem is a dictionary of genes' expression folds, e.g.:

In [ ]:
solution = {'b3956': 8, 'b2914': 4}

You may get information on the genes

In [ ]:
sim.find_genes('b3956|b2914')

The gene expressions are converted by the problem into reaction constraints:

In [ ]:
constraints = problem.solution_to_constraints(solution)
constraints

that are used to run phenotypic simulations:

In [ ]:
sim.simulate(constraints=constraints).find()

This functionality allows, for example, to simulate a same genetic modification on different methods or using other GEMs.
You may also use directly the problem instance to simulate the modifications:

In [ ]:
problem.simulate(solution=solution,method='pFBA').find(['succ','BIOMASS'])

In [ ]:
problem.FVA('EX_succ_e', solution=solution, format='df')